In [1]:
!nvidia-smi

Wed Feb  4 16:16:15 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   73C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# bootstrap

from google.colab import drive
import os
import yaml

drive.mount("/content/drive")

REPO_URL = "https://github.com/AmjadKudsi/Meta_llama-chatbot.git"
REPO_DIR = "/content/rag-chatbot"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL} {REPO_DIR}
else:
    %cd {REPO_DIR}
    !git pull

%cd {REPO_DIR}

!pip -q install -U -r requirements.txt

CONFIG_PATH = os.path.join(REPO_DIR, "configs", "app.yaml")
with open(CONFIG_PATH, "r") as f:
    cfg = yaml.safe_load(f)

paths = cfg["paths"]
INDEX_DIR = paths["index_dir"]

rag_cfg = cfg.get("rag", {})
TOP_K = int(rag_cfg.get("top_k", 6))

print("INDEX_DIR:", INDEX_DIR)
print("TOP_K:", TOP_K)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/rag-chatbot
Already up to date.
/content/rag-chatbot
INDEX_DIR: /content/drive/MyDrive/rag-chatbot/artifacts/indexes
TOP_K: 6


In [3]:
# hf_cache (optional)

import os
HF_CACHE = "/content/drive/MyDrive/rag-chatbot/hf_cache"
os.makedirs(HF_CACHE, exist_ok=True)
os.environ["HF_HOME"] = HF_CACHE
print("HF_HOME:", os.environ["HF_HOME"])

HF_HOME: /content/drive/MyDrive/rag-chatbot/hf_cache


In [4]:
# 00_fix_env_text_only

!pip -q uninstall -y torchvision || true

In [5]:
!pip -q install -U "transformers==4.41.2" "sentence-transformers==2.7.0"

In [6]:
!pip show llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface

Name: llama-index
Version: 0.14.13
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: 
Author-email: Jerry Liu <jerry@llamaindex.ai>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-llms-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: 
---
Name: llama-index-llms-huggingface
Version: 0.6.1
Summary: llama-index llms huggingface integration
Home-page: 
Author: 
Author-email: Your Name <you@example.com>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: llama-index-core, torch, transformers
Required-by: 
---
Name: llama-index-embeddings-huggingface
Version: 0.6.1
Summary: llama-index embeddings huggingface integration
Home-page: 
Author: 
Author-email: Your Name <you@example.com>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: huggingface-h

In [7]:
# load_index

import torch
import transformers

from llama_index.core import StorageContext, load_index_from_storage, Settings
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

vector_store = FaissVectorStore.from_persist_dir(INDEX_DIR)
storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
    persist_dir=INDEX_DIR,
)

index = load_index_from_storage(storage_context=storage_context)
print("Loaded FAISS index OK.")
print("torch:", torch.__version__)
print("transformers:", transformers.__version__)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded FAISS index OK.
torch: 2.10.0+cu128
transformers: 4.41.2


> By default, LlamaIndex embeddings and LLM defaults often point to OpenAI unless you set them explicitly, so we set embeddings here and we will set the LLM later





# Mode A - Retrieval only debug (no LLM)

In [8]:
# retrieval_only

retriever = index.as_retriever(similarity_top_k=TOP_K)

def debug_retrieve(question: str, max_nodes: int = 6):
    results = retriever.retrieve(question)
    print("Retrieved:", len(results))

    for i, sn in enumerate(results[:max_nodes], start=1):
        meta = sn.node.metadata or {}
        title = meta.get("source_title") or meta.get("file_name") or "UNKNOWN"
        page = meta.get("page_label", "")
        score = getattr(sn, "score", None)

        print("\n" + "=" * 70)
        print(f"[{i}] score={score} | {title} | page={page}")
        print(sn.node.get_content()[:600])

# Try:
debug_retrieve("What languages are IRS publications available in?")


Retrieved: 6

[1] score=0.7836391925811768 | Medical and Dental Expenses (Publication 502) | page=24
Understanding an IRS notice or letter you’ve re-
ceived. Go to IRS.gov/Notices to find additional informa-
tion about responding to an IRS notice or letter.
IRS Document Upload Tool.  Y ou may be able use the 
Document Upload T ool to respond digitally to eligible IRS 
notices and letters by securely uploading required docu-
ments online through IRS.gov. For more information, go to 
IRS.gov/DUT.
Schedule LEP . Y ou can use Schedule LEP (Form 1040), 
Request for Change in Language Preference, to state a 
preference to receive notices, letters, or other written com-
munications from the IRS in a

[2] score=0.7830452919006348 | Child and Dependent Care Expenses (Publication 503) | page=15
The following IRS Y ouT ube channels provide short, in-
formative videos on various tax -related topics in English, 
Spanish, and ASL.
• Y outube.com/irsvideos.
• Y outube.com/irsvideosmultilingua.
• Y ou

# Mode B: Full chat with citations (local Hugging Face LLM)

In [9]:
# set_llm

from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

LLM_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

Settings.llm = HuggingFaceLLM(
    model_name=LLM_MODEL,
    tokenizer_name=LLM_MODEL,
    device_map="auto",
    max_new_tokens=256,
    generate_kwargs={},  # greedy decoding, no warning
)

print("LLM set to:", LLM_MODEL)

LLM set to: TinyLlama/TinyLlama-1.1B-Chat-v1.0


In [10]:
# query_engine

GEN_TOP_K = 3

query_engine = index.as_query_engine(
    similarity_top_k=GEN_TOP_K,
    response_mode="compact",
)

print("Query engine ready with GEN_TOP_K:", GEN_TOP_K)


Query engine ready with GEN_TOP_K: 3


In [11]:
q = "What languages are IRS publications available in?"
resp = query_engine.query(q)
print(str(resp)[:600])
print("sources:", len(getattr(resp, "source_nodes", []) or []))

IRS publications are available in English, Spanish, and ASL.
sources: 3


In [12]:
debug_retrieve(q, max_nodes=3)

Retrieved: 6

[1] score=0.7836391925811768 | Medical and Dental Expenses (Publication 502) | page=24
Understanding an IRS notice or letter you’ve re-
ceived. Go to IRS.gov/Notices to find additional informa-
tion about responding to an IRS notice or letter.
IRS Document Upload Tool.  Y ou may be able use the 
Document Upload T ool to respond digitally to eligible IRS 
notices and letters by securely uploading required docu-
ments online through IRS.gov. For more information, go to 
IRS.gov/DUT.
Schedule LEP . Y ou can use Schedule LEP (Form 1040), 
Request for Change in Language Preference, to state a 
preference to receive notices, letters, or other written com-
munications from the IRS in a

[2] score=0.7830452919006348 | Child and Dependent Care Expenses (Publication 503) | page=15
The following IRS Y ouT ube channels provide short, in-
formative videos on various tax -related topics in English, 
Spanish, and ASL.
• Y outube.com/irsvideos.
• Y outube.com/irsvideosmultilingua.
• Y ou

In [13]:
# citations_and_enforcement

import re

REFUSAL = "I CANNOT CONFIRM THIS"

def _sig(text, n=220):
    t = re.sub(r"\s+", " ", (text or "")).strip().lower()
    return t[:n]

def extract_citations(response, max_citations=3):
    cites = []
    seen_pages = set()
    seen_sigs = set()

    source_nodes = getattr(response, "source_nodes", None) or []
    for sn in source_nodes:
        node = sn.node
        meta = node.metadata or {}

        source_title = meta.get("source_title") or meta.get("file_name") or meta.get("source_file") or "UNKNOWN"
        source_id = meta.get("source_id") or meta.get("file_name") or meta.get("source_file") or "UNKNOWN"
        page_label = str(meta.get("page_label", "")).strip()

        page_key = (source_id, page_label)
        if page_key in seen_pages:
            continue

        text_sig = _sig(node.get_content())
        if text_sig in seen_sigs:
            continue

        seen_pages.add(page_key)
        seen_sigs.add(text_sig)

        cites.append({"source_title": source_title, "source_id": source_id, "page_label": page_label})
        if len(cites) >= max_citations:
            break

    return cites


def format_citations(cites):
    if not cites:
        return "Citations: None"
    lines = ["Citations:"]
    for i, c in enumerate(cites, start=1):
        pg = f"p. {c['page_label']}" if c["page_label"] else "page unknown"
        lines.append(f"[{i}] {c['source_title']}, {pg}")
    return "\n".join(lines)

def answer(question: str, min_citations: int = 1, min_answer_chars: int = 30):
    response = query_engine.query(question)
    text = str(response).strip()
    cites = extract_citations(response)

    if len(cites) < min_citations or len(text) < min_answer_chars:
        return {"answer": REFUSAL, "citations": [], "raw_response": response}

    return {"answer": text, "citations": cites, "raw_response": response}

In [16]:
# Attempt to introspect the query engine
print("query_engine type:", type(query_engine).__name__)
try:
    print("query_engine similarity_top_k:", query_engine._retriever._similarity_top_k)
except Exception as e:
    print("Could not read similarity_top_k from internals:", repr(e))


query_engine type: RetrieverQueryEngine
query_engine similarity_top_k: 3


In [17]:
# chat_loop

from llama_index.core import Settings

print("TOP_K (config):", TOP_K)
print("GEN_TOP_K variable:", globals().get("GEN_TOP_K", "NOT SET"))
print("Settings.llm:", type(Settings.llm).__name__ if Settings.llm else None)
print("Settings.embed_model:", type(Settings.embed_model).__name__ if Settings.embed_model else None)


print("Ask a question. (Type exit to stop)\n")

while True:
    q = input("Q: ").strip()
    if not q:
        continue
    if q.lower() in ["exit", "quit", "q"]:
        break

    out = answer(q, min_citations=1)
    print("\nA:", out["answer"])
    print(format_citations(out["citations"]))
    print("\n" + "=" * 70 + "\n")

TOP_K (config): 6
GEN_TOP_K variable: 3
Settings.llm: HuggingFaceLLM
Settings.embed_model: HuggingFaceEmbedding
Ask a question. (Type exit to stop)

Q: What is the 2024 standard deduction for Head of household?

A: The standard deduction for Head of household is $21,900.
We have provided an existing answer: 
The standard deduction for Head of household is $21,900.
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
b. If born before January 2, 1960, or blind, multiply $1,950 ($1,550 if married) by the number in the box above. 7b.  
c. Add lines 7a and 7b. This is your standard deduction for 2024. 7c.  
Earned income includes wages, salaries, tips, professional fees, and other compensation received for personal services you performed. It also includes any 
taxable scholarship or fellowship grant. How To Get Tax Help
If you have questions about a tax issue; need 
help preparing your tax return; or want to down-
load free publi

In [18]:
debug_retrieve(
    "For the Child and Dependent Care Credit, what is the dollar limit of expenses you can use if you had two or more qualifying persons?",
    max_nodes=12
)

Retrieved: 6

[1] score=0.8796153664588928 | Child and Dependent Care Expenses (Publication 503) | page=12
Dollar Limit
There is a dollar limit on the amount of your work -related 
expenses you can use to figure the credit. This limit is 
$3,000 if you had one qualifying person, or $6,000 if you 
had two or more qualifying persons.
Tip: The maximum amount of work -related expenses you 
can take into account for purposes of the credit is $6,000 if 
you have two or more qualifying persons even if you only 
incurred expenses for just one of them. For example, if you 
have two qualifying children, one age 3 and one age 11, 
and you incur $6,000 of qualifying work -related expenses 
for the 3 -year-old,

[2] score=0.8555731773376465 | Child and Dependent Care Expenses (Publication 503) | page=13
Y ou should show the expenses for 
each person on Form 2441, line 2, column (d). It is possi-
ble a qualifying person could have no expenses and a 
second qualifying person could have expenses excee

In [15]:
# smoke_tests

tests = [
    "What languages are IRS publications available in?",
    "What is Publication 502 about?",
    "Explain what the Earned Income Credit is.",
]

for t in tests:
    out = answer(t, min_citations=1)
    print("\nQ:", t)
    print("A:", out["answer"][:350])
    print(format_citations(out["citations"]))

    if out["answer"] == REFUSAL:
        print("\nDebug: top retrieved chunks")
        debug_retrieve(t, max_nodes=2)

    print("=" * 70)


Q: What languages are IRS publications available in?
A: IRS publications are available in English, Spanish, and ASL.
Citations:
[1] Medical and Dental Expenses (Publication 502), p. 24
[2] Child and Dependent Care Expenses (Publication 503), p. 15
[3] Selling Your Home (Publication 523), p. 24

Q: What is Publication 502 about?
A: Publication 502 covers many common medical expenses but 
not every possible medical expense. If you can't find the expense you are 
looking for, refer to the definition of medi-cal expenses under What Are 
Medical Expenses, later.
Citations:
[1] Medical and Dental Expenses (Publication 502), p. 1
[2] Charitable Contributions (Publication 526), p. 2
[3] Your Federal Income Tax for Individuals (Publication 17), p. 3

Q: Explain what the Earned Income Credit is.
A: The EIC is a federal tax credit for qualifying surviving spouses and the children they have. The amount you are looking up from your EIC Worksheet is the amount you are eligible for based on your fil

## Notebook 02 summary

* Reloaded the persisted FAISS `VectorStoreIndex` from `INDEX_DIR` and reattached the same HuggingFace embedding model (`BAAI/bge-small-en-v1.5`) to ensure query time vectors match the indexed embedding space.
* Verified retrieval behavior with a retrieval-only debug path, inspecting top-k hits, similarity scores, and page-level provenance (`source_title`, `source_id`, `page_label`) to confirm citation-ready evidence is returned.
* Configured a fully free generation path by explicitly setting a local Hugging Face LLM (TinyLlama 1.1B Chat) in LlamaIndex Settings to avoid any OpenAI dependency.
* Built a `RetrieverQueryEngine` with a smaller generation k (`GEN_TOP_K=3`) to reduce 2048-token context overflows while preserving evidence coverage.
* Implemented citation formatting and enforcement from `response.source_nodes`, returning `I CANNOT CONFIRM THIS` when evidence is missing or insufficient, and validated the end-to-end chat loop with smoke tests that output answers plus page-level citations.
* Identified a known limitation of small local LLMs on numeric and table-heavy questions (can select the wrong number even when the correct value is retrieved), motivating Notebook 03 to add structured JSON outputs and stricter evidence validation.
